# <center>Big Data for Engineers – Solutions</center>
## <center>Spring 2020 – Week 8 – ETH Zurich</center>
## <center>YARN</center>

# What is YARN?


Fundamentally, “**Y**et **A**nother **R**esource **N**egotiator”. **YARN**  is a resource management systems designed to work on existing and new Hadoop clusters. 

YARN supports pluggable schedulers. The task of the scheduler is to share the resources of a large cluster among different tenants (applications) while trying to meet application demands (memory, CPU). A user may have several applications active at a time. 

As mentioned in the lecture, this week's exercise will **not** focus on how the scheduling happens inside YARN, i.e., in which order and how the apllications are launched. This is out of the scope of this lecture. The important content of this exercise is the architecture of YARN and how the resource manager, application masters and namenodes interact. 

### 0 (Warm up) – Different resources
Before starting with the exercises on YARN we will recap the different resources that can be requested in a cluster. Remeber that in MapReduce v1 each map and reduce task are processed in map and reduce slots. These slots are bundles of resources. In the lecture we saw that the in YARN the solts are called containers. 

Name the different resources that we can request. 

### 0 (Warm up) – Solution

We can request for different amounts of Memory (i.e., RAM) and CPU (nr. cores). The option to request disk space and  network bandwidth is still work in progress. Therefore, we usually define the resources by the need of memory and cores. 

### 1 – List at least 3 main shortcomings of MapReduce v1, which are addressed by YARN design.

### 1 – Solution
1. Scalability. YARN can scale to 10,000 nodes.
2. Need for non-MapReduce workloads. Ability to share cluster with MPI, graph processing, and any user code.
3. Better resource utilization. Problem with idle reducers, when mappers are not finished (and vice versa). The entire system should be used at maximum capacity when needed.
4. Not flexible, mapper and reducer decided at configuration time
5. Agility. Ability to maintain MapReduce frameworks of different versions.

### 2 – Name at least 4 characteristics that should be fulfilled in YARN. 
(Properties not mentioned in 1)

### 2 – Solution

1. Scalability
2. Multi-tenancy (Multiple clients share same application and hardware resources)
3. Serviceability
4. Locality awareness (for performance)
5. High cluster utilization
6. Reliability/Availability
7. Secure and auditable operation
8. Support for programming model diversity (i.e. not only MapReduce style
communication and programming patterns)
9. Flexible resource model (to be able to change allocated amount of resources
to a job while running)
10. Backwards compatibility

### 3 – State which of the following statements are true:

1. The ResourceManager has to provide fault tolerance for resources across the cluster 

1. Container allocation/deallocation can take place in a dynamic fashion as the application progresses. 

1. YARN plans to allow applications to only request resources in terms of memory usage and number of CPUs.

1. Communications between the ResourceManager and NodeManagers are heartbeat-based. 

1. The ResourceManager does not have a global view of all usage of cluster resources. Therefore, it tries to make better scheduling decisions based on probabilistic prediction. 

1. ResourceManager has the ability to request resources back from a running application.

### 3 – Solution
1. **False**

1. **True**

1. **False**

1. **True**

1. **False**

1. **True**

### 4 – Whose responsibility is it? Say which component of YARN is resposible for each of the following tasks (can be multiple).

1. Fault Tolerance of running applications *\[ResourceManager | ApplicationMaster | NodeManager \]*

1. Holding a two lists of valid and invalid Nodes. *\[ResourceManager | ApplicationMaster | NodeManager\]*

1. Asking for resources needed for an application *\[ResourceManager | ApplicationMaster | NodeManager \]*

1. Providing leases to use containers *\[ResourceManager | ApplicationMaster | NodeManager\]*

1. Tracking status and progress of running applications *\[ResourceManager | ApplicationMaster | NodeManager\]*

1. Offers a log aggregation service that will upload data written by the application to stdout and stderr to HDFS once the application
completes. *\[ResourceManager | ApplicationMaster | NodeManager\]*

1. Interacts with admin and client services. *\[ResourceManager | ApplicationMaster | NodeManager\]*

1. Tracking and reporting status of containers *\[ResourceManager | ApplicationMaster | NodeManager\]*

1. Authentication of container leases. *\[ResourceManager | ApplicationMaster | NodeManager\]*



### 4 – Solution

1. Fault Tolerance of running applications *\[ResourceManager | **ApplicationMaster** | NodeManager \]*

1. Holding a two lists of valid and invalid Nodes. *\[**ResourceManager** | ApplicationMaster | NodeManager\]*

1. Asking for resources needed for an application *\[ResourceManager | **ApplicationMaster** | NodeManager \]*

1. Providing leases to use containers *\[**ResourceManager** | ApplicationMaster | NodeManager\]*

1. Tracking status and progress of running applications *\[ResourceManager | **ApplicationMaster** | NodeManager\]*

1. Offers a log aggregation service that will upload data written by the application to stdout and stderr to HDFS once the application
completes. *\[ResourceManager | ApplicationMaster | **NodeManager**\]*

1. Interacts with admin and client services. *\[**ResourceManager** | ApplicationMaster | NodeManager\]*

1. Tracking and reporting status of containers *\[ResourceManager | **ApplicationMaster** | **NodeManager**\]*

1. Authentication of container leases. *\[ResourceManager | ApplicationMaster | **NodeManager**\]*



### 5 – What is the typical configuration for YARN? Choose for the following components how many instances of them there are in a cluster.

```
1. ResourceManager                  a. One per cluster

2. ApplicationMaster                b. One per node

3. NodeManager                      c. Many per cluster, but usually not per every node

4. Container                        d. Many per node 
```

### 5 – Solution
1 - a  
2 - c  
3 - b  
4 - d  

Note that we can have multiple ApplicationMasters on the same node as long as they belong to different applications and none of them is aware about the presence of the other ApplicationMasters.

### 6 – What are ResourceRequests? What are they used for?

### 6 – Solution

ApplicationMasters codify their need for resources in
terms of one or more ResourceRequests, each of which
tracks:
1. number of containers (e.g., 200 containers),
2. resources8 per container h2GB RAM, 1 CPUi,
3. locality preferences, and
4. priority of requests within the application

ResourceRequests are designed to allow users to capture
the full detail of their needs and/or a roll-up version
of it (e.g., one can specify node-level, rack-level, and
global locality preferences). We see that the possibility to decide locality preferences satisfies is a key requirement for YARN to be locality aware.

### 7 – What is a container launch context (ClC)? What are they used for?

### 7 – Solution

All containers in YARN– including AMs– are described
by a container launch context (CLC). This record includes a map of environment variables, dependencies stored in remotely accessible storage, security tokens, payloads for NM services, and the command
necessary to create the process.

YARN application writers submitt the application by passing a CLC or the ApplicationMaster to the Resource manager. 

### 8 – Client Resource Request 
The following task concerns the process of a client requesting resources and it from the optional reading material (Murthy, A. C. (2014). Apache Hadoop YARN: Moving beyond MapReduce and Batch Processing with Apache Hadoop 2., Chapter 4). Remember that several frameworks can run on top of YARN not only MapReduce. 

A YARN application starts with a client resource request. 


Match the following steps to the arrows in the picture below:
<ol type="a">
  <li>The ResourceManager will send information about the minimum and maximum capabilities of the cluster.</li>
  <li>The ResourceManager responds to the ApplicationMaster with its assigned resources.</li>
  <li>The client performs an application request to the Resource Manager.</li>
  <li>The application master requests a number of containers.</li>
  <li>The Resource Manager responds to the client with an ApplicationID with information needed to start the ApplicationMaster.</li>
  <li>The client sends to the ResourceManager an Application Submission Context about its requirements and other necessary info.</li>
  <li>The ResourceManager contacts the NodeManager to start the client Application Master and sends a Registration Request.</li>
</ol>

![](https://bigdata2020exassets.blob.core.windows.net/bd4e2021ex08/ClientResourceRequest.png)











### 8 – Solution

Step 1) --> c.

Step 2) --> e.

Step 3) --> f.

Step 4) --> g.

Step 5) --> a.

Step 6) --> d.

Step 7) --> b.